## Esercizio
(esercizio 2.2 dell'esercitazione del 31 ottobre)

Scrivere una funzione `flat : btree -> int list list` che restituisca una lista contenente le liste di valori presenti ad ogni livello di profondità dell’albero.

In [1]:
type btree =
| Void
| Node of int * btree * btree;;

let bt = Node (3,
  Node (5,
    Node(1,Void,Void),
    Void
  ),
  Node (-4,
    Node(6,Void,Void),
    Node(8,Void,Void)
  )
);;

type btree = Void | Node of int * btree * btree


val bt : btree =
  Node (3, Node (5, Node (1, Void, Void), Void),
   Node (-4, Node (6, Void, Void), Node (8, Void, Void)))


In [2]:
(*     l1 = [ [5]    ; [1]     ]
       l2 = [ [-4]   ; [6;8]   ]
aux l1 l2 = [ [5;-4] ; [1;6;8] ]
*)

let rec aux l1 l2 =
  match l1, l2 with
    | [], _ -> l2
    | _, [] -> l1
    | (x::xs), (y::ys) -> (x@y)::(aux xs ys);;

aux [[5]; [1]] [[-4]; [6;8]];;

let rec flat bt =
  match bt with
    | Void -> []
    (* | Node (v, Void, Void) -> [v] *)
    | Node (v, lt, rt) -> let ll = flat lt in let rl = flat rt in
        [v]::(aux ll rl);;

flat bt;;

val aux : 'a list list -> 'a list list -> 'a list list = <fun>


- : int list list = [[5; -4]; [1; 6; 8]]


val flat : btree -> int list list = <fun>


- : int list list = [[3]; [5; -4]; [1; 6; 8]]


## Esercizio
(esercizio 3 esame dicembre 2021)

In [3]:
type 'a mset_lst = 'a list;;
type 'a mset_map = ('a * int) list;;

type 'a mset_lst = 'a list


type 'a mset_map = ('a * int) list


In [4]:
(* Calcola la molteplicità di e nel multiset m, rappresentato come mset_lst *)
let rec mult_lst (m : 'a mset_lst) (e: 'a) =
  match m with
    | [] -> 0
    | x::xs -> if x = e then 1 + mult_lst xs e else mult_lst xs e;;

(*
Con funzioni higher-order

let mult_lst (m : 'a mset_lst) (e: 'a) =
  List.len (List.filter (fun x -> x = e) m)
  
  List.foldl (fun acc x -> if x = e then 1 + acc else acc) 0 m
*)

mult_lst [2;4;3;2;2;3] 2;;

(* Calcola la molteplicità di e nel multiset m, rappresentato come mset_map *)
let rec mult_map (m: 'a mset_map) e = 
  match m with
    | [] -> 0
    | (x, n)::rest -> if x = e then n else mult_map rest e;;

(*
Con funzioni higher-order
let mult_map m e =
  match List.find (fun (x, n) -> x = e) m with
    | (_, n) -> n

*)

mult_map [('a',3);('c',1);('b',2)] 'a'

let rec contained e l =
  match l with
    | [] -> false
    | (x, _)::rest -> if x = e then true else contained e rest;;

let rec remove (e : 'a) (l : ('a * int) list) =
  match l with
    | [] -> (0, [])
    | (x, n)::rest -> if x = e then (n, rest) else
        let (m, rest') = remove e rest in (m, ((x, n)::rest'));;

remove 'b' [('a',3);('c',1);('b',2)];;

(* Unione di multiset *)
let sum m1 m2 =
  let rec aux partial_result m =
    match m with
      | [] -> partial_result
      | (x, n)::rest -> if contained x partial_result
        then
          let (m, partial_res') = remove x partial_result in
          aux ((x, n + m)::partial_res') rest
        else
          aux ((x, n)::partial_result) rest
  in
    aux (aux [] m1) m2;;

let fst (x, _) = x

let sum m1 m2 =
  (* prende ['a'; 'b'] dalla prima lista e poi ci concatena la seconda lista a cui toglie i duplicati
     applica una List.map (fun x -> (x, mult_map m1 x + mult_map m2 x))
  *)
  let tmp1 = List.map fst m1 in
  let tmp = tmp1@(List.filter (fun x -> not (List.mem x tmp1)) (List.map fst m2)) in
  List.map (fun x -> (x, mult_map m1 x + mult_map m2 x)) tmp;;

sum [('a',2);('b',3)] [('b',4);('c',1)];;

val mult_lst : 'a mset_lst -> 'a -> int = <fun>


- : int = 3


val mult_map : 'a mset_map -> 'a -> int = <fun>


- : int = 3


val contained : 'a -> ('a * 'b) list -> bool = <fun>


...

- : int * (char * int) list = (2, [('a', 3); ('c', 1)])


val sum : ('a * int) list -> ('a * int) list -> ('a * int) list = <fun>


val fst : 'a * 'b -> 'a = <fun>


val sum : 'a mset_map -> 'a mset_map -> ('a * int) list = <fun>


- : (char * int) list = [('a', 2); ('b', 7); ('c', 1)]


## Esercizio
(esercizio 4 dell'esame di giugno 2022)

Estendere MiniCaml con un costrutto cfun.

Come tutti gli esercizi dell'interprete, procediamo in 4 passaggi.

### Espressioni

In [5]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TCfun | TUnBound

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* Costrutto cfun *)
    | ECfun of ide * exp * ide * exp * ide * exp


type ide = string


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TCfun
  | TUnBound


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | ECfun of ide * exp * ide * exp * ide * exp


### Valori esprimibili

In [6]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | CFun of ide * exp * ide * exp * ide * exp * evT env
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | CFun of ide * exp * ide * exp * ide * exp * evT env
  | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


### Typechecking

In [7]:
(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | CFun(_,_,_,_,_,_,_) -> TCfun
    | UnBound -> TUnBound

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TCfun -> (match y with
                | CFun(_,_,_,_,_,_,_) -> true
                | _ -> false
                )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

val getType : evT -> tname = <fun>


val typecheck : tname * evT -> bool = <fun>


### Interprete

In [8]:
(* Errori a runtime *)
exception RuntimeError of string

(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

exception RuntimeError of string


val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


...

val less_than : evT * evT -> evT = <fun>


val greater_than : evT * evT -> evT = <fun>


val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


In [9]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            (* Nuovo caso Apply(CFun, exp) *)
            | CFun(px, pBody, f1x, f1Body, f2x, f2Body, cclosure) ->
                let argVal = eval eArg s in
                let pEnv = bind cclosure px argVal in
                let guard = eval pBody pEnv in
                (match (typecheck(TBool,guard), guard) with
                | (true, Bool(true)) ->
                  let f1Env = bind cclosure f1x argVal in
                    eval f1Body f1Env
                | (true, Bool(false)) ->
                  let f2Env = bind cclosure f2x argVal in
                    eval f2Body f2Env
                | (_, _) -> raise ( RuntimeError "Wrong type")
                )
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (* Nuovo costrutto ECfun *)
    | ECfun(px, pBody, f1x, f1Body, f2x, f2Body) ->
        CFun(px, pBody, f1x, f1Body, f2x, f2Body, s)

val eval : exp -> evT env -> evT = <fun>


In [10]:
let exp1 = Apply(
    ECfun (
        "x", GreaterThan (Den "x", EInt 0),
        "y", Sum (Den "y", EInt 1),
        "z", Sum (Den "z", EInt 10)
    ),
    EInt (-20)
);;

eval exp1 emptyenv;;

val exp1 : exp =
  Apply
   (ECfun ("x", GreaterThan (Den "x", EInt 0), "y", Sum (Den "y", EInt 1),
     "z", Sum (Den "z", EInt 10)),
   EInt (-20))


- : evT = Int (-10)
